In [14]:
import cupy as cp
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from io import StringIO

# Dados Sintéticos Realistas (Binance) - Adaptado para leitura de CSV
binance_csv = """Altcoin,Preço (USD),Volume (24h),Market Cap (USD),Variação (24h)
Binance Coin (BNB),215,400000000,34000000000,1.2
Cardano (ADA),0.27,180000000,8500000000,-1.5
Solana (SOL),20.5,350000000,7000000000,2.1
Dogecoin (DOGE),0.062,200000000,8500000000,0.5
Polkadot (DOT),4.25,150000000,7800000000,-0.2
Shiba Inu (SHIB),0.000008,100000000,4500000000,3.0
Polygon (MATIC),0.65,80000000,5500000000,-0.7
Avalanche (AVAX),11.2,120000000,4000000000,1.8"""

df_altcoins_binance = pd.read_csv(StringIO(binance_csv))

# Dados (GitHub)
github_csv = """Altcoin,Número de Commits (último mês),Número de Issues Abertas,Número de Pull Requests (último mês),Atividade da Comunidade (escala 1-10)
Cosmos (ATOM),250,35,80,9
Polkadot (DOT),200,40,70,8
Filecoin (FIL),120,20,40,7
Avalanche (AVAX),180,30,60,8.5
Chainlink (LINK),150,25,50,7.5
Solana (SOL),300,50,100,9.5
Cardano (ADA),220,45,85,8.5
Polygon (MATIC),280,55,90,9
Algorand (ALGO),170,32,65,7.8"""

df_altcoins_github = pd.read_csv(StringIO(github_csv))

# Função para plotar gráfico de barras
def plot_bar_chart(df, x_col, y_col, title):
    fig = px.bar(df, x=x_col, y=y_col, title=title,
                 color_discrete_sequence=px.colors.qualitative.Dark24)
    fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

# Função para plotar gráfico de pizza
def plot_pie_chart(df, values, names, title):
    fig = px.pie(df, values=values, names=names, title=title,
                 color_discrete_sequence=px.colors.qualitative.Dark24)
    fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

# Função para plotar boxplot
def plot_boxplot(df, x_col, y_col, title):
    fig = px.box(df, x=x_col, y=y_col, title=title,
                 color_discrete_sequence=px.colors.qualitative.Dark24)
    fig.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

# Função para plotar matriz de correlação
def plot_correlation_matrix(df, title):
    numeric_df = df.select_dtypes(include=[np.number])
    corr_matrix = numeric_df.corr()
    fig = go.Figure(data=go.Heatmap(z=corr_matrix.values,
                                   x=corr_matrix.index,
                                   y=corr_matrix.columns,
                                   colorscale='Viridis'))
    fig.update_layout(title=title,
                      xaxis_nticks=36,
                      plot_bgcolor='black', paper_bgcolor='black', font_color='white')
    fig.show()

# Visualizações dos Dados de Binance
plot_bar_chart(df_altcoins_binance, 'Altcoin', 'Preço (USD)', 'Preço das Altcoins (USD)')
plot_pie_chart(df_altcoins_binance, 'Market Cap (USD)', 'Altcoin', 'Market Cap das Altcoins')
plot_boxplot(df_altcoins_binance, 'Altcoin', 'Volume (24h)', 'Distribuição do Volume (24h)')

# Visualizações dos Dados de GitHub
plot_bar_chart(df_altcoins_github, 'Altcoin', 'Número de Commits (último mês)', 'Número de Commits por Altcoin')
plot_correlation_matrix(df_altcoins_github, 'Matriz de Correlação - Dados GitHub')

# Exemplo de uso das funções originais (opcional)
# df = generate_altcoin_data()
# plot_time_series(df)
# plot_pca(df)